In [108]:
from functional.backtester import Backtester as b
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from modeler_strats.universal_modeler import UniversalModeler
from transformer.adhoc_transforms import Adhoc as adhocs
from transformer.risk_transforms import Risk as risks
from transformer.analysis_transforms import Analysis as anas

import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
from statistics import variance
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm

In [109]:
start_date = datetime(2023,1,1)
end_date = datetime.now()

In [110]:
market = Market()
sec = SEC()
fin_db = ADatabase("financial")
earnings_db = ADatabase("earnings")
dividends_db = ADatabase("dividends")
speculation_db = ADatabase("spec")
classification_db = ADatabase("spec_classification")
current_db = ADatabase("financial_current")
umod = UniversalModeler()

In [111]:
tyields = adhocs.tyields()
bench_returns = adhocs.spy_bench()

In [112]:
market.connect()
sp500 = market.retrieve("sp500")
sp500 = sp500.rename(columns={"Symbol":"ticker"})
market.disconnect()

In [113]:
new_prices = []
market.connect()
sec.connect()
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        cik = int(sp500[sp500["ticker"]==ticker]["CIK"])
        financials = sec.retrieve_filing_data(cik)
        ticker_sim = market.retrieve_ticker_prices("prices",ticker)
        completed = risks.quarterly_risk_prep(ticker_sim,bench_returns,financials)
        new_prices.append(completed)
    except:
        continue
sec.disconnect()
market.disconnect()
price_returns = pd.concat(new_prices)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [02:59<00:00,  2.74it/s]


In [114]:
values = [True,False]
ceilings = [True,False]
parameters = []
positions = 10
strats = ["dividends","earnings","sector","financials"]
for strat in strats:
    for value in values:
        for ceiling in ceilings:
            parameter = {"value":value
                         ,"ceiling":ceiling
                        ,"strat":strat
                        }
            parameters.append(parameter)

In [115]:
fin_db.connect()
financial_simulation = fin_db.retrieve("predictions")
fin_db.disconnect()

In [116]:
earnings_db.connect()
earnings_simulation = earnings_db.retrieve("predictions")
earnings_db.disconnect()

In [117]:
dividends_db.connect()
dividends_simulation = dividends_db.retrieve("predictions")
dividends_db.disconnect()

In [118]:
sim = price_returns.merge(tyields[["year","quarter","quarterly_yield"]],on=["year","quarter"],how="left") \
                    .merge(financial_simulation[["year","quarter","ticker","financial_prediction"]],on=["year","quarter","ticker"],how="left") \
                    .merge(earnings_simulation[["year","quarter","ticker","earnings_prediction"]],on=["year","quarter","ticker"],how="left") \
                    .merge(dividends_simulation[["year","quarter","ticker","dividends_prediction"]],on=["year","quarter","ticker"],how="left")
sim = sim.dropna().groupby(["year","quarter","ticker"]).mean().reset_index()

In [119]:
ranks = sim.merge(sp500[["ticker","GICS Sector"]],how="left").groupby(["year","quarter","GICS Sector"]).mean().reset_index().sort_values(["year","quarter","quarterly_return"],ascending=False).groupby(["year","quarter"]).first().reset_index().rename(columns={"GICS Sector":"top_sector"})[["year","quarter","top_sector"]]
ranks["year"] = ranks["year"] + 1

In [120]:
price_returns

,year,quarter,ticker,quarter_start,quarter_end,return_end,commonstockdividendspersharecashpaid,quarterly_return,bench_quarterly_return,quarterly_variance,market_quarterly_cov,quarterly_beta
56,2013,2,MMM,78.359545,81.56865,0.040954,0.6125,0.048770,-0.020945,0.000270,0.000000e+00,0.000000e+00
57,2013,2,MMM,78.359545,81.56865,0.040954,0.6125,0.048770,-0.008415,0.000258,1.445603e-19,5.608449e-16
58,2013,2,MMM,78.359545,81.56865,0.040954,0.6125,0.048770,-0.006358,0.000250,0.000000e+00,0.000000e+00
59,2013,2,MMM,78.359545,81.56865,0.040954,0.6125,0.048770,0.018888,0.000366,-7.228014e-20,-1.975455e-16
60,2013,2,MMM,78.359545,81.56865,0.040954,0.6125,0.048770,0.033132,0.000559,-7.228014e-20,-1.293619e-16
...,...,...,...,...,...,...,...,...,...,...,...,...
2708,2023,2,ZTS,167.300266,164.74000,-0.015303,0.0000,-0.015303,-0.025784,0.000055,-2.818926e-18,-5.153302e-14
2709,2023,2,ZTS,167.300266,164.74000,-0.015303,0.0000,-0.015303,-0.029570,0.000041,-2.818926e-18,-6.942096e-14
2710,2023,2,ZTS,167.300266,164.74000,-0.015303,0.0000,-0.015303,-0.027023,0.000032,-2.891206e-18,-9.029543e-14
2711,2023,2,ZTS,167.300266,164.74000,-0.015303,0.0000,-0.015303,-0.021594,0.000031,-2.891206e-18,-9.276014e-14


In [121]:
sim.sort_values(["year","quarter"],inplace=True)

In [122]:
dividend_tickers = adhocs.dividend_tickers(price_returns)

In [123]:
current_db.connect()
current_db.drop("trades")
for parameter in tqdm(parameters):
    simulation = sim.copy()
    strat = parameter["strat"]
    simulation = risks.quarterly_backtest_prep(strat,simulation,dividend_tickers,sp500,ranks)
    b.quarterly_backtest(simulation.copy(),positions,parameter,start_date,end_date,current_db)
current_db.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 17.12it/s]
